In [1]:
import json
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.manifold import MDS, TSNE, LocallyLinearEmbedding, Isomap
from MulticoreTSNE import MulticoreTSNE
from sklearn.decomposition import TruncatedSVD, PCA, KernelPCA, SparsePCA
import umap
import pandas as pd
from gb_writer import GlyphboardWriter
# from sklearn.model_selection import GridSearchCV
from typing import Any
import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam
# import spacy
# from spacy.lang.de.stop_words import STOP_WORDS


# nlp = spacy.load('de')

Using TensorFlow backend.


In [2]:
def getTestData():
    with open("test_data.json", "r") as read_file:
        test_data = json.load(read_file)
    test_texts = []
    test_labels = []
    for doc in test_data:
    #     # only use confident labels
         if (doc["features"]["1"]["4"] > 0.5):
            test_labels.append(1)
            test_texts.append(doc["values"]["7"])
         else:
             test_labels.append(0)
             test_texts.append(doc["values"]["7"])
    return pd.DataFrame({'text': test_texts, 'label': test_labels})

def train(train_data, test_data, algo: Any) -> dict:
    # clean_data = [preprocessText(text) for text in data]
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        # ('nlp', preprocessText())
        # ('clf', LogisticRegression()),
        # ('clf', MultinomialNB()),
        ('clf', algo),
    ])
    text_clf.fit(train_data.text, train_data.label)
    # text_clf.fit(clean_data, labels)
    predicted = text_clf.predict(test_data.text)
    # print(dataframe.prediction.value_counts())
    # print(metrics.classification_report(test_labels, predicted))
    addHistory(metrics.f1_score(test_data.label, predicted))
    result = {
        # 'prediction': predicted.toList(),
        # 'accuracy': metrics.accuracy_score(test_labels, predicted),
        'precision': metrics.precision_score(test_data.label, predicted),
        'recall': metrics.recall_score(test_data.label, predicted),
        'f1': metrics.f1_score(test_data.label, predicted),
        'f1_history': getHistory()
    }
    return result

def getHistory():
    history = []
    with open(
            "mlbackend/metrics.csv", "r", encoding="utf8") as file:
        reader = csv.reader(file, delimiter=';')
        for line in reader:
            history.append(line[0])
        file.close()
    return history

def addHistory(metrics):
    with open(
            "mlbackend/metrics.csv", "a",  newline="", encoding="utf8") as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow([str(metrics)])
        file.close()

In [3]:
# getTestData()

In [4]:
def applyDR(tfidf, labels):
    # mds = MDS(n_components=2, random_state=1).fit_transform(tfidf.toarray())
    lsi = TruncatedSVD(n_components=100, random_state=1).fit_transform(tfidf.toarray())
    # mds = MDS(n_components=2, random_state=1).fit_transform(lsi)
    labels = labels.reshape(len(labels), 1)
    with_labels = np.hstack((lsi,labels))
    with_labels.shape
    mds = MDS(n_components=2, random_state=1).fit_transform(with_labels)
#     mds = TSNE(n_components=2, random_state=1).fit_transform(with_labels)
    df = pd.DataFrame(columns=['x', 'y'])
    df['x'] = mds[:, 0]
    df['y'] = mds[:, 1]
    print(df)
    return df

In [5]:
data = getTestData()
vec = TfidfVectorizer()
tfidf = vec.fit_transform(data.text)

In [25]:
lsi = TruncatedSVD(n_components=2, random_state=1).fit_transform(tfidf.toarray())
# lsi = PCA(n_components=10, random_state=1).fit_transform(tfidf.toarray())

labels = np.asarray(data.label)
labels = labels.reshape(len(labels), 1)
with_labels = np.hstack((lsi,labels))



# m = Sequential()
# m.add(Dense(512,  activation='elu', input_shape=(101,)))
# m.add(Dense(128,  activation='elu'))
# m.add(Dense(2,    activation='linear', name="bottleneck"))
# m.add(Dense(128,  activation='elu'))
# m.add(Dense(512,  activation='elu'))
# m.add(Dense(101,  activation='sigmoid'))
# m.compile(loss='mean_squared_error', optimizer = Adam())
# history = m.fit(with_labels, with_labels, batch_size=128, epochs=5, verbose=1)

# encoder = Model(m.input, m.get_layer('bottleneck').output)
# lsi_with_labels = encoder.predict(with_labels)

# lsi_with_labels = PCA(n_components=2, random_state=1).fit_transform(with_labels)
# lsi_with_labels = TruncatedSVD(n_components=2, random_state=1).fit_transform(with_labels)
# lsi_with_labels = Isomap(n_components=2).fit_transform(with_labels)
# lsi_with_labels = MulticoreTSNE(n_jobs=1, n_components=2, random_state=1).fit_transform(with_labels)
# lsi_with_labels = LocallyLinearEmbedding(n_jobs=4,n_components=2, random_state=1).fit_transform(with_labels)
lsi_with_labels = umap.UMAP(random_state=1).fit_transform(lsi)
df = pd.DataFrame(columns=['x', 'y'])
df['x'] = lsi_with_labels[:, 0]
df['y'] = lsi_with_labels[:, 1]
    
    
df *= 1
# DR.x *= 500
# DR.y *= 500

print(df)

writer = GlyphboardWriter('test_name')

# DR *= 2
print('Writing positions...')    
writer.write_position(df, 'lsi')
del df
print('done')

             x         y
0     7.506356  1.401674
1    -3.612138 -2.039203
2     0.478525 -5.534179
3    -4.284943  2.448015
4     0.889897 -1.649368
5    -4.903683 -3.059757
6    -6.948209  0.938133
7     0.536590 -6.534554
8    -6.884484  7.115571
9    -3.871984 -4.557209
10    7.921367  0.130527
11   -0.959576  5.622871
12    6.262791 -4.693450
13    6.628671 -2.361881
14   -7.039403  6.142621
15    1.305112 -6.200580
16    7.474166 -3.319695
17   -6.099412  2.795892
18   -0.830204 -6.461778
19   -7.006938  0.670415
20   -1.570214 -2.628725
21    3.324758 -6.201512
22    1.430066 -6.286922
23   -0.806018  5.878713
24    6.487830 -0.327867
25   -1.429619 -3.714862
26   -5.294108 -0.819282
27    3.279274 -5.504846
28    7.839597  1.452218
29    0.370711 -3.867836
...        ...       ...
1088 -4.631029 -2.740127
1089 -0.455486 -3.913275
1090 -1.507559 -4.510925
1091  6.201534 -4.752929
1092 -6.304682  5.091229
1093 -2.570395 -4.474161
1094  0.967407 -4.657150
1095  6.079129 -4.846052


In [32]:
with open("test_data.json", "r") as read_file:
        test_data = json.load(read_file)
test_texts = []
test_labels = []
ids = []

for doc in test_data:
    ids.append(doc['id'])
    test_labels.append(1)
    test_texts.append(doc["values"]["7"])

df = pd.DataFrame({'id': ids, 'text': test_texts, 'label': test_labels})
df.to_csv('data.csv', sep=";")
# return df